# Welcome to the ASX share performance benchmarking tool.
- Please feel free to change the following variables to interact with this notebook.
- The following blocks will produce benchmarking results in command-line messages and interactive charts. 

In [240]:
import investpy
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime

Start date in DD/MM/YYYY

In [241]:
start_date = "01/01/2020"

Enter an ASX ticker

In [242]:
ticker_1 = "VHY"

Enter another ASX ticker

In [243]:
ticker_2 = "VAS"

### Fetching Share Price Data

In [244]:
end_date = datetime.date.today().strftime("%d/%m/%Y")
ticker_countries = ["australia"]

try:
    share_1 = investpy.search_quotes(text = ticker_1, countries = ticker_countries, n_results = 1)
    share_1 = share_1.retrieve_historical_data(from_date = start_date, to_date = end_date)[["Close"]]

    share_2 = investpy.search_quotes(text = ticker_2, countries = ticker_countries, n_results = 1)
    share_2 = share_2.retrieve_historical_data(from_date = start_date, to_date = end_date)[["Close"]]
except ValueError:
    print("Incorrect date format. Please try again.")
except RuntimeError:
    print("Ticker not found. Please try again.")
except:
    print("An error has occured. Please try again.")

### Performance Benchmark Results

In [245]:
bench_share_1 = (share_1["Close"] / share_1.iloc[0]["Close"] - 1) * 100
bench_share_2 = (share_2["Close"] / share_2.iloc[0]["Close"] - 1) * 100

print(ticker_1, "has gained", str(round(bench_share_1.iloc[-1] * 100, 2)) + "%", "since", start_date)
print(ticker_2, "has gained", str(round(bench_share_2.iloc[-1] * 100, 2)) + "%", "since", start_date)

performance_bench_chart = go.Figure()
performance_bench_chart.add_trace(go.Scatter(x = share_1.index, y = bench_share_1, name = ticker_1))
performance_bench_chart.add_trace(go.Scatter(x = bench_share_2.index, y = bench_share_2, name = ticker_2))
performance_bench_chart.update_layout(title = ticker_1 + " vs. " + ticker_2 + " since " + start_date + " (%)")
performance_bench_chart.show()

VHY has gained 678.36% since 01/01/2020
VAS has gained 185.38% since 01/01/2020


### Monetary Outperformance/Underperformance

In [246]:
performance_diff = pd.DataFrame(share_1["Close"] - share_2["Close"] - (share_1.iloc[0]["Close"] - share_2.iloc[0]["Close"]))
performance_diff.rename(columns = {"Close": "Difference $"}, inplace = True)
diff_plot = px.area(performance_diff, y = "Difference $", title = ticker_1 + " performance against " + ticker_2 + " since " + start_date + " ($)")
diff_plot.show()

### Percentage Outperformance/Underperformance

In [247]:
performance_pct_diff = pd.DataFrame(performance_diff["Difference $"] / share_2["Close"] * 100)
performance_pct_diff.rename(columns = {0: "Difference %"}, inplace = True)

diff_pct_plot = px.area(performance_pct_diff, y = "Difference %", title = ticker_1 + " performance against " + ticker_2 + " since " + start_date + " (%)")
diff_pct_plot.show()

### Combined Chart

In [248]:
performance_bench_chart = go.Figure()
performance_bench_chart.add_trace(go.Scatter(x = performance_pct_diff.index, y = performance_pct_diff["Difference %"], fill = 'tonexty', name = "Performance Difference %"))
performance_bench_chart.add_trace(go.Scatter(x = share_1.index, y = bench_share_1, name = ticker_1))
performance_bench_chart.add_trace(go.Scatter(x = bench_share_2.index, y = bench_share_2, name = ticker_2))
performance_bench_chart.update_layout(title = ticker_1 + " vs. " + ticker_2 + " since " + start_date + " (%)")
performance_bench_chart.show()